#### Сбор таблиц

In [1]:
from collections import Counter
import json
import os
import re
from tqdm import tqdm

import numpy as np
import pandas as pd

from requests import get
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import seaborn as sns
import shap

%matplotlib inline

In [2]:
# страницы с перечислением таблиц, для которых указан SEQN

webpages = ['https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Questionnaire&Cycle=2017-2020',
            'https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Examination&Cycle=2017-2020',
            'https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Demographics&Cycle=2017-2020',
            'https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Laboratory&Cycle=2017-2020']

In [3]:
# Собираем ссылки на страницы с файлами таблиц и их описанием

tables = []
descriptions = []

for wp in webpages:
    webpage = get(wp).text
    hrefs = [(tag.get_text(), tag['href']) 
             for tag in BeautifulSoup(webpage, 'html.parser').find_all('a', href=True)]
    hrefs = hrefs[hrefs.index(('SAS Universal Viewer', '/nchs/nhanes/sasviewer.aspx')) + 1:]
    for href in hrefs:
        field_name = href[0]
        link = href[1]
        if 'Doc' in field_name:
            descriptions += ['https://wwwn.cdc.gov' + link]
        elif 'Data' in field_name:
            tables += ['https://wwwn.cdc.gov' + link]
        else:
            break

In [4]:
# проверяем, что количество страниц совпадает

len(descriptions) == len(tables)

True

In [8]:
# Скачиваем все таблицы, кроме P_DIQ.XPT, так как ее признаки нельзя использовать ни в качестве предикторов, 
# ни в качестве лейблов, в корневую папку. Скачаем P_DIQ.XPT в отдельную папку 

for table in tqdm(tables):
    r = get(table, allow_redirects=True)
    table_name = table.split('/')[-1]
    if table_name == 'P_DIQ.XPT':
        if not os.path.isdir('./P_DIQ'):
            os.mkdir('P_DIQ')
        open(f'./P_DIQ/{table_name}', 'wb').write(r.content)
    open(table_name, 'wb').write(r.content)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 73/73 [12:02<00:00,  9.89s/it]


In [9]:
check_df = pd.read_sas(f'./P_DIQ/P_DIQ.XPT')

In [10]:
# Создадим отдельно набор данных с таблицами, признаки которых нельзя использовать в качестве предикторов. 
# Вероятно, именно их стоит использовать в качестве лейблов. 
# Проверим это на таблице, которую нельзя использовать.

tables_for_label = ['P_GHB.XPT', 'P_GLU.XPT']
label_df = pd.merge(pd.read_sas(tables_for_label[0]), pd.read_sas(tables_for_label[1]), 
                    how='outer', on='SEQN')

In [11]:
label_df.head(10)

,SEQN,LBXGH,WTSAFPRP,LBXGLU,LBDGLUSI
0,109264.0,5.3,27533.174559,97.0,5.38
1,109266.0,5.2,NaN,NaN,NaN
2,109271.0,5.6,18100.965319,103.0,5.72
3,109273.0,5.1,NaN,NaN,NaN
4,109274.0,5.7,16281.758327,154.0,8.55
5,109277.0,5.3,32230.046209,92.0,5.11
6,109279.0,5.0,NaN,NaN,NaN
7,109282.0,5.5,79007.100787,95.0,5.27
8,109284.0,5.6,NaN,NaN,NaN
9,109285.0,NaN,NaN,NaN,NaN


In [12]:
# Посмотрим, для каких SEQN меньше всего информации

label_df['n_of_not_nan'] = label_df.apply(lambda x: sum(~x.isna()), axis=1)
label_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10409 entries, 0 to 10408
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SEQN          10409 non-null  float64
 1   LBXGH         9737 non-null   float64
 2   WTSAFPRP      5090 non-null   float64
 3   LBXGLU        4744 non-null   float64
 4   LBDGLUSI      4744 non-null   float64
 5   n_of_not_nan  10409 non-null  int64  
dtypes: float64(5), int64(1)
memory usage: 569.2 KB


In [13]:
label_df.describe().drop('count').style.bar()

,SEQN,LBXGH,WTSAFPRP,LBXGLU,LBDGLUSI,n_of_not_nan
mean,117094.509367,5.765575,53883.958149,111.182125,6.171739,3.335959
std,4500.406208,1.066217,68538.125759,36.314648,2.016121,1.533076
min,109264.000000,2.800000,0.000000,47.000000,2.610000,1.000000
25%,113190.000000,5.200000,17305.966829,95.000000,5.270000,2.000000
50%,117114.000000,5.500000,32845.099026,102.000000,5.660000,2.000000
75%,120999.000000,5.900000,62135.880230,112.000000,6.220000,5.000000
max,124822.000000,16.200000,741259.188749,524.000000,29.100000,5.000000


In [14]:
Counter(label_df['n_of_not_nan'])

Counter({5: 4732, 2: 5261, 1: 359, 4: 12, 3: 45})

In [15]:
pd.merge(check_df, label_df, on='SEQN', how='left').head(10)

,SEQN,DIQ010,DID040,DIQ160,DIQ180,DIQ050,DID060,DIQ060U,DIQ070,DIQ230,...,DID341,DID350,DIQ350U,DIQ360,DIQ080,LBXGH,WTSAFPRP,LBXGLU,LBDGLUSI,n_of_not_nan
0,109263.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,109264.0,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.3,27533.174559,97.0,5.38,5.0
2,109265.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,109266.0,2.0,NaN,1.0,1.0,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,5.2,NaN,NaN,NaN,2.0
4,109267.0,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,109268.0,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,109269.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,109270.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,109271.0,2.0,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.6,18100.965319,103.0,5.72,5.0
9,109273.0,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.1,NaN,NaN,NaN,2.0


In [16]:
# Загрузим описания признаков таблиц и соберем их в словари

features_descr_dict = {}
table_features_dict = {}

for descr in tqdm(descriptions):
    webpage = get(descr).text
    #wrong_titles = 'English Instructions: '
    titles_descr = [tag.get_text() for tag in BeautifulSoup(webpage, 'html.parser').find_all('dt')]
    if not titles_descr:
        continue
    titles = [tag.get_text() for tag in BeautifulSoup(webpage, 'html.parser').find_all('dd')]
    table_name_code = [tag.get_text() for tag in BeautifulSoup(webpage, 'html.parser').find_all('h3')][1]
    table_name_code, table_name = '('.join(table_name_code.split('(')[:-1]), table_name_code.split('(')[-1][:-1]
    table_years = re.findall('(\d+ YEARS)', titles[3])
    table_years_diff = int(table_years[1].split()[0]) - int(table_years[0].split()[0])
    
    # хочется данные для разных возрастов. Если разница возрастов 
    if table_years_diff > 20:
        for title, descr in zip(titles, titles_descr):
            if descr == 'Variable Name: ':
                feature_name = title
                if feature_name not in features_descr_dict:
                    features_descr_dict[feature_name] = {}
                features_descr_dict[feature_name]['question'] = []
                if table_name not in table_features_dict:
                    table_features_dict[table_name] = [feature_name]
                else:
                    table_features_dict[table_name] += [feature_name]
            elif descr == 'SAS Label: ':
                feature_label = title
                features_descr_dict[feature_name]['label'] = feature_label
            
            elif descr in ['English Text: ', 'English Instructions: ']:
                feature_question = title
                features_descr_dict[feature_name]['question'] += [feature_question]
            

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 73/73 [02:45<00:00,  2.27s/it]


In [17]:
# сохраняем словари

with open("table_features_dict.json", "w") as outfile:
    json.dump(table_features_dict, outfile)
    
with open("features_descr_dict.json", "w") as outfile:
    json.dump(features_descr_dict, outfile)

In [18]:
# Смотрим, что можно взять в качестве целевой переменной

for feature in table_features_dict['P_DIQ']:
    print(feature)
    if 'label' in features_descr_dict[feature]:
        print(features_descr_dict[feature]['label'])

SEQN
Respondent sequence number
DIQ010
Doctor told you have diabetes
DID040
Age when first told you had diabetes
DIQ159
DIQ160
Ever told you have prediabetes
DIQ180
Had blood tested past three years
DIQ050
Taking insulin now
DID060
How long taking insulin
DIQ060U
Unit of measure (month/year)
DIQ065
DIQ070
Take diabetic pills to lower blood sugar
DIQ229
DIQ230
How long ago saw a diabetes specialist
DIQ240
Is there one Dr you see for diabetes
DID250
Past year how many times seen doctor
DID260
How often check blood for glucose/sugar
DIQ260U
Unit of measure (day/week/month/year)
DIQ275
Past year Dr checked for A1C
DIQ280
What was your last A1C level
DIQ291
What does Dr say A1C should be
DIQ295
DIQ300S
What was your recent SBP
DIQ300D
What was your recent DBP
DID310S
What does Dr say SBP should be
DID310D
What does Dr say DBP should be
DID320
What was most recent LDL number
DID330
What does Dr say LDL should be
DID341
Past year times Dr check feet for sores
DID350
How often do you check you

In [19]:
# диабет и преддиабет - 'DIQ010', 'DIQ160' - объединяем с label_df 

predict_labels = pd.merge(check_df[['SEQN', 'DIQ010', 'DIQ160']], label_df, on='SEQN', how='left')
predict_labels_diab = predict_labels.query('(DIQ010 == 1 or DIQ160 == 1) and n_of_not_nan > 0').copy()
predict_labels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14986 entries, 0 to 14985
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SEQN          14986 non-null  float64
 1   DIQ010        14986 non-null  float64
 2   DIQ160        9516 non-null   float64
 3   LBXGH         9737 non-null   float64
 4   WTSAFPRP      5090 non-null   float64
 5   LBXGLU        4744 non-null   float64
 6   LBDGLUSI      4744 non-null   float64
 7   n_of_not_nan  10409 non-null  float64
dtypes: float64(8)
memory usage: 1.0 MB


In [20]:
# перекодируем значения

for i in predict_labels_diab[~predict_labels_diab.DIQ160.isna()].index:
    predict_labels.loc[i, 'DIQ010'] = .5
    
print('Значения для преддиабета: ', predict_labels[~predict_labels.DIQ160.isna()].DIQ010.unique())
print('Значения для диабета: ', predict_labels.DIQ010.unique())

predict_labels.DIQ010 = predict_labels.DIQ010.map({2.:0., 0.5:0.5, 1.:1., 3.:0.5, 9.:9.})
predict_labels = predict_labels[predict_labels.DIQ010 != 9.]

print('Перекодированные значения: ', predict_labels.DIQ010.unique())

Значения для преддиабета:  [2.  0.5 9. ]
Значения для диабета:  [2.  0.5 1.  3.  9. ]
Перекодированные значения:  [0.  0.5 1. ]


In [21]:
# оставляем строки, для которых есть целевая переменная

predict_labels = predict_labels[predict_labels.n_of_not_nan > 0]
predict_labels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10405 entries, 1 to 14985
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SEQN          10405 non-null  float64
 1   DIQ010        10405 non-null  float64
 2   DIQ160        8800 non-null   float64
 3   LBXGH         9733 non-null   float64
 4   WTSAFPRP      5087 non-null   float64
 5   LBXGLU        4741 non-null   float64
 6   LBDGLUSI      4741 non-null   float64
 7   n_of_not_nan  10405 non-null  float64
dtypes: float64(8)
memory usage: 731.6 KB
